In [3]:
import json
import mysql.connector
import nvda_config
import requests
import pandas as pd

from datetime import datetime, timedelta


In [5]:
# set up login
Cpassword = nvda_config.mysql_password
api_k = nvda_config.finhub_api
finnhub_base_url = 'https://finnhub.io/api/v1'


In [8]:
mydb = mysql.connector.connect(
    host="localhost",  
    user="root",         
    password=Cpassword,  
    database="nvdav1"   
)
mycursor = mydb.cursor()

In [13]:
# Date range for historical data (adjust as needed)
end_date = datetime.now()
start_date = end_date - timedelta(days=365)  # Get the past year's data

# Convert dates to Unix timestamps (milliseconds)
end_timestamp = int(end_date.timestamp() * 1000)
start_timestamp = int(start_date.timestamp() * 1000)

# Get stock ticks
tick_endpoint = f'/stock/tick?symbol=NVDA&from={start_timestamp}&to={end_timestamp}&token={api_k}'
tick_url = finnhub_base_url + tick_endpoint
tick_response = requests.get(tick_url)
tick_data = json.loads(tick_response.content)
print(tick_data)
# Insert tick data into MySQL
for tick in tick_data['t']:
    sql = "INSERT INTO stock_ticks (symbol, time, price, volume) VALUES (%s, %s, %s, %s)"
    values = ("NVDA", tick, tick_data['p'][tick], tick_data['v'][tick])
    mycursor.execute(sql, values)

mydb.commit()

print(mycursor.rowcount, "records inserted.")

{'error': "You don't have access to this resource."}
{'error': "You don't have access to this resource."}


KeyError: 't'